# YOLO12n-SEG Inference

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
!nvcc --version

import os
import torch
from IPython import get_ipython


BASE_PATH = os.path.dirname(os.getcwd())
VISIBLE_GPUS = [3,4,6]

os.environ["CUDA_VISIBLE_DEVICES"] = str(VISIBLE_GPUS).replace("]", "").replace("[","").replace(" ", "")
os.environ["NVIDIA_VISIBLE_DEVICES"] = str(VISIBLE_GPUS).replace("]", "").replace("[","").replace(" ", "")
print("CUDA GPUS NUMBER: ", torch.cuda.device_count())

In [ ]:
from ultralytics import YOLO

model = YOLO('/model_weights/YOLO11n-seg/best.pt')
PROJECT_PATH = '/path/to/your/project'

YAML_RELATIVE_PATH = "dataset/data.yaml"
DATA_YAML = os.path.join(BASE_PATH, YAML_RELATIVE_PATH)


print(BASE_PATH)
print(DATA_YAML)
print(YAML_RELATIVE_PATH)

In [5]:
import numpy as np
import json
import os

class NPEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super().default(obj)

In [ ]:
metrics_test = model.val(save_json=True, project=PROJECT_PATH, name="test", split="test")
metrics_val = model.val(save_json=True, project=PROJECT_PATH, name="val", split="val")


summary_json = {
    "metrics": metrics_val.results_dict,
    "speed": metrics_val.speed,
}
summary_json["metrics"].update({"map50": metrics_val.seg.map50})
summary_json["metrics"].update({"map75": metrics_val.seg.map75})
summary_json["metrics"].update({"f1": metrics_val.seg.f1})

with open(os.path.join(PROJECT_PATH, "val/val_summary.json"), "w") as of:
    json.dump(summary_json, of, cls=NPEncoder)

summary_json = {
    "metrics": metrics_test.results_dict,
    "speed": metrics_test.speed,
}
summary_json["metrics"].update({"map50": metrics_test.seg.map50})
summary_json["metrics"].update({"map75": metrics_test.seg.map75})
summary_json["metrics"].update({"f1": metrics_test.seg.f1})

with open(os.path.join(PROJECT_PATH, "test/test_summary.json"), "w") as of:
    json.dump(summary_json, of, cls=NPEncoder)


In [19]:
print(metrics_test.seg.map50)
print(metrics_test.seg.map75)
print(metrics_val.seg.map50)
print(metrics_val.seg.map75)

0.38669334000399624
0.2993233137098712
0.381434346529227
0.3023487525323844


In [11]:
print(metrics_test.seg.map50)
print(metrics_test.seg.map75)
print(metrics_val.seg.map50)
print(metrics_val.seg.map75)

0.38669334000399624
0.2993233137098712
0.381434346529227
0.3023487525323844
